In [2]:
import os

In [3]:
%pwd

'd:\\projectFiles\\text_summarization_project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\projectFiles\\text_summarization_project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

    

In [7]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml,create_directories


In [8]:
CONFIG_FILE_PATH = "D:\\projectFiles\\text_summarization_project\\config\\config.yaml"
PARAMS_FILE_PATH = "D:\\projectFiles\\text_summarization_project\\params.yaml"

class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

        # Convert the file paths to pathlib.Path objects
        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)

        # Print the paths when initializing
        print("Config File Path:", config_filepath)
        print("Params File Path:", params_filepath)

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        root_dir = Path(config.root_dir)


        create_directories([root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config


In [9]:
from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_from_disk
import torch
from transformers import BartTokenizer
from transformers import BartForConditionalGeneration


In [27]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt,force_download=True, resume_download=False).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # loading data
        dataset_samsum_pt = load_from_disk(str(self.config.data_path))

        # Select only 10 values from each split (train, validation, test)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=50,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=50, save_steps=1e6,
            gradient_accumulation_steps=40
        )

        trainer = Trainer(
            model=model_pegasus, args=trainer_args,
            tokenizer=tokenizer, data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        # Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [28]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

Config File Path: D:\projectFiles\text_summarization_project\config\config.yaml
Params File Path: D:\projectFiles\text_summarization_project\params.yaml
[2024-01-02 04:22:30,192: INFO: common:yaml file: D:\projectFiles\text_summarization_project\config\config.yaml loaded successfully]
[2024-01-02 04:22:30,210: INFO: common:yaml file: D:\projectFiles\text_summarization_project\params.yaml loaded successfully]
[2024-01-02 04:22:30,214: INFO: common:created directory at: artifacts]
[2024-01-02 04:22:30,219: INFO: common:created directory at: artifacts\model_trainer]


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

OSError: Unable to load weights from pytorch checkpoint file for 'C:\Users\satya\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\snapshots\40d588fdab0cc077b80d950b300bf66ad3c75b92\pytorch_model.bin' at 'C:\Users\satya\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\snapshots\40d588fdab0cc077b80d950b300bf66ad3c75b92\pytorch_model.bin'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.